In [1]:
import pandas as pd
from modeling_old import clear_column_names, random_search

In [2]:
def load_data(path="./df_2ap_final.csv", threshold=1):
    df = pd.read_csv(path, header=[0, 1], index_col=None)

    y = df[("mcs_nss", "_")]
    X = df.drop(columns=[("mcs_nss", "_")])

    X.columns = clear_column_names(X)

    # 计算每个类别的频次
    value_counts = y.value_counts()
    # 找到频次大于阈值的类别
    valid_categories = value_counts[value_counts >= threshold].index

    # 过滤样本
    mask = y.isin(valid_categories)
    X = X[mask]
    y = y[mask]

    return X, y


X, y = load_data(threshold=5)  # 设置阈值为 5
X.shape, y.shape

((382, 212), (382,))

In [3]:
from sklearn.preprocessing import LabelEncoder

# 创建 LabelEncoder 实例
le = LabelEncoder()

# 转换目标变量
y_encoded = le.fit_transform(y)

# to pd.Series
y_encoded = pd.Series(y_encoded)

In [4]:
y_encoded.value_counts(), len(y_encoded.value_counts())

(1    249
 2     48
 7     23
 0     23
 4     16
 6     11
 3      7
 5      5
 Name: count, dtype: int64,
 8)

In [5]:
model_types = dict(
    # random_forest=dict(
    #     n_estimators=[50, 100, 200],
    #     max_features=[None, "sqrt", "log2"],
    #     max_depth=[None, 10, 20],
    # ),
    # extra_trees=dict(
    #     n_estimators=[50, 100, 200],
    #     max_features=[None, "sqrt", "log2"],
    #     max_depth=[None, 10, 20],
    # ),
    # xgboost=dict(
    #     objective="multi:softmax",  # 多分类
    #     num_class=15,  # 类别数量
    #     n_estimators=[50, 100, 200],
    #     max_depth=[None, 10, 20],
    #     learning_rate=[0.01, 0.1, 0.2],
    # ),
    # lightgbm=dict(
    #     objective="multiclass",  # 多分类
    #     metric="multi_logloss",  # 多分类的损失函数
    #     n_estimators=[50, 100, 200],
    #     learning_rate=[0.01, 0.1, 0.2],
    #     max_depth=[None, 10, 20],
    #     num_leaves=31,
    #     min_data_in_leaf=20,
    #     feature_fraction=0.9,
    #     early_stopping_round=10,
    # ),
    # catboost=dict(
    #     iterations=[50, 100, 200],
    #     depth=[4, 6, 8],
    #     learning_rate=[0.01, 0.1, 0.2],
    #     l2_leaf_reg=[1, 3, 5],
    #     loss_function="MultiClass",  # 多分类
    #     bootstrap_type=["Bernoulli", "MVS"],
    # ),
    mlp=dict(
        hidden_size1=[64, 128, 256],
        hidden_size2=[32, 64, 128],
        weight_decay=[0.001, 0.0001, 1e-5],
        output_size=14,  # 类别数量
    ),
)

In [6]:
results = {}
for k, v in model_types.items():
    print(f"Running {k}")
    res, _ = random_search(
        param_dict=v, X=X, y=y_encoded, model_type=k, suffix="2ap", n_trials=10
    )
    results[k] = res

Running mlp

Params: {'hidden_size1': 256, 'hidden_size2': 32, 'weight_decay': 0.001, 'output_size': 14}


/data/twh/code/huaweicup/q2/modeling_old.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.replace({True: 1, False: 0})


Epoch [20/600], Loss: 1.906666
Epoch [40/600], Loss: 1.859807
Epoch [60/600], Loss: 1.813035
Epoch [80/600], Loss: 1.808187
Epoch [100/600], Loss: 1.803601
Epoch [120/600], Loss: 1.803317
Epoch [140/600], Loss: 1.796233
Epoch [160/600], Loss: 1.786399
Epoch [180/600], Loss: 1.786016
Epoch [200/600], Loss: 1.779615
Epoch [220/600], Loss: 1.778117
Epoch [240/600], Loss: 1.776846
Epoch [260/600], Loss: 1.775275
Epoch [280/600], Loss: 1.774805
Epoch [300/600], Loss: 1.774513
Epoch [320/600], Loss: 1.775226
Epoch [340/600], Loss: 1.774162
Epoch [360/600], Loss: 1.775087
Epoch [380/600], Loss: 1.774880
Epoch [400/600], Loss: 1.775114
Epoch [420/600], Loss: 1.775598
Epoch [440/600], Loss: 1.775678
Epoch [460/600], Loss: 1.773702
Epoch [480/600], Loss: 1.773847
Epoch [500/600], Loss: 1.775149
Epoch [520/600], Loss: 1.773138
Epoch [540/600], Loss: 1.771699
Epoch [560/600], Loss: 1.773539
Epoch [580/600], Loss: 1.771564
Epoch [600/600], Loss: 1.771319


/data/twh/code/huaweicup/q2/modeling_old.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.replace({True: 1, False: 0})


Epoch [20/600], Loss: 1.793040
Epoch [40/600], Loss: 1.776381
Epoch [60/600], Loss: 1.772267
Epoch [80/600], Loss: 1.771360
Epoch [100/600], Loss: 1.771916
Epoch [120/600], Loss: 1.771379
Epoch [140/600], Loss: 1.769893
Epoch [160/600], Loss: 1.769866
Epoch [180/600], Loss: 1.771649
Epoch [200/600], Loss: 1.770275
Epoch [220/600], Loss: 1.769583
Epoch [240/600], Loss: 1.769647
Epoch [260/600], Loss: 1.770543
Epoch [280/600], Loss: 1.769732
Epoch [300/600], Loss: 1.770560
Epoch [320/600], Loss: 1.770630
Epoch [340/600], Loss: 1.769687
Epoch [360/600], Loss: 1.770644
Epoch [380/600], Loss: 1.771508
Epoch [400/600], Loss: 1.769781
Epoch [420/600], Loss: 1.769671
Epoch [440/600], Loss: 1.769607
Epoch [460/600], Loss: 1.771568
Epoch [480/600], Loss: 1.770624
Epoch [500/600], Loss: 1.770588
Epoch [520/600], Loss: 1.769698
Epoch [540/600], Loss: 1.769728
Epoch [560/600], Loss: 1.769655
Epoch [580/600], Loss: 1.769681
Epoch [600/600], Loss: 1.769666


...

Epoch [20/600], Loss: 1.886826
Epoch [40/600], Loss: 1.868063
Epoch [60/600], Loss: 1.837438
Epoch [80/600], Loss: 1.827573
Epoch [100/600], Loss: 1.820860
Epoch [120/600], Loss: 1.815573
Epoch [140/600], Loss: 1.796231
Epoch [160/600], Loss: 1.797339
Epoch [180/600], Loss: 1.779951
Epoch [200/600], Loss: 1.780637
Epoch [220/600], Loss: 1.778787
Epoch [240/600], Loss: 1.779609
Epoch [260/600], Loss: 1.779622
Epoch [280/600], Loss: 1.778833
Epoch [300/600], Loss: 1.778861
Epoch [320/600], Loss: 1.779652
Epoch [340/600], Loss: 1.778827
Epoch [360/600], Loss: 1.778792
Epoch [380/600], Loss: 1.780521
Epoch [400/600], Loss: 1.779628
Epoch [420/600], Loss: 1.780469
Epoch [440/600], Loss: 1.780458
Epoch [460/600], Loss: 1.779593
Epoch [480/600], Loss: 1.779642
Epoch [500/600], Loss: 1.778747
Epoch [520/600], Loss: 1.778727
Epoch [540/600], Loss: 1.781287
Epoch [560/600], Loss: 1.778778
Epoch [580/600], Loss: 1.780434
Epoch [600/600], Loss: 1.778740


/data/twh/code/huaweicup/q2/modeling_old.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.replace({True: 1, False: 0})


Epoch [20/600], Loss: 1.897714
Epoch [40/600], Loss: 1.829415
Epoch [60/600], Loss: 1.803280
Epoch [80/600], Loss: 1.798036
Epoch [100/600], Loss: 1.791555
Epoch [120/600], Loss: 1.791778
Epoch [140/600], Loss: 1.788159
Epoch [160/600], Loss: 1.789752
Epoch [180/600], Loss: 1.787904
Epoch [200/600], Loss: 1.789468
Epoch [220/600], Loss: 1.788164
Epoch [240/600], Loss: 1.788880
Epoch [260/600], Loss: 1.787644
Epoch [280/600], Loss: 1.787283
Epoch [300/600], Loss: 1.786094
Epoch [320/600], Loss: 1.781957
Epoch [340/600], Loss: 1.780689
Epoch [360/600], Loss: 1.781836
Epoch [380/600], Loss: 1.779715
Epoch [400/600], Loss: 1.779576
Epoch [420/600], Loss: 1.779451
Epoch [440/600], Loss: 1.780291
Epoch [460/600], Loss: 1.779465
Epoch [480/600], Loss: 1.780339
Epoch [500/600], Loss: 1.781231
Epoch [520/600], Loss: 1.779503
Epoch [540/600], Loss: 1.781101
Epoch [560/600], Loss: 1.781963
Epoch [580/600], Loss: 1.780265
Epoch [600/600], Loss: 1.781124


/data/twh/code/huaweicup/q2/modeling_old.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.replace({True: 1, False: 0})


Epoch [20/600], Loss: 1.905045
Epoch [40/600], Loss: 1.841563
Epoch [60/600], Loss: 1.820246
Epoch [80/600], Loss: 1.815210
Epoch [100/600], Loss: 1.807402
Epoch [120/600], Loss: 1.793541
Epoch [140/600], Loss: 1.779834
Epoch [160/600], Loss: 1.771064
Epoch [180/600], Loss: 1.771111
Epoch [200/600], Loss: 1.771174
Epoch [220/600], Loss: 1.769962
Epoch [240/600], Loss: 1.770059
Epoch [260/600], Loss: 1.770880
Epoch [280/600], Loss: 1.770878
Epoch [300/600], Loss: 1.770851
Epoch [320/600], Loss: 1.770077
Epoch [340/600], Loss: 1.770866
Epoch [360/600], Loss: 1.770862
Epoch [380/600], Loss: 1.770861
Epoch [400/600], Loss: 1.770008
Epoch [420/600], Loss: 1.770901
Epoch [440/600], Loss: 1.770856
Epoch [460/600], Loss: 1.770838
Epoch [480/600], Loss: 1.771742
Epoch [500/600], Loss: 1.770043
Epoch [520/600], Loss: 1.769998
Epoch [540/600], Loss: 1.770017
Epoch [560/600], Loss: 1.770892
Epoch [580/600], Loss: 1.770114
Epoch [600/600], Loss: 1.770004
mlp model does not support feature importanc